# Лабораторная работа номер 6

Выполнил: Стрельцов Илья Дмитриевич Б9122-01.03.02мкт

In [21]:
import kagglehub
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler


In [4]:
PATH = "mczielinski/bitcoin-historical-data"
SEED = 5

## Загрузка датасета с историей биткоина

In [5]:
path = kagglehub.dataset_download(PATH)

Resuming download from 14680064 bytes (88207552 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/mczielinski/bitcoin-historical-data?dataset_version_number=425 (14680064/102887616) bytes left.


100%|██████████| 98.1M/98.1M [11:08<00:00, 132kB/s] 

Extracting files...


In [12]:
from pathlib import Path

file_path = Path(path) / 'btcusd_1-min_data.csv'
df = pd.read_csv(file_path)

## Обзорная информация о датасете

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7304797 entries, 0 to 7304796
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Timestamp  float64
 1   Open       float64
 2   High       float64
 3   Low        float64
 4   Close      float64
 5   Volume     float64
dtypes: float64(6)
memory usage: 334.4 MB


In [14]:
df

,Timestamp,Open,High,Low,Close,Volume
0,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
1,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
2,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
3,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
4,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
...,...,...,...,...,...,...
7304792,1.763769e+09,85065.00,85087.00,85031.00,85087.00,3.943960
7304793,1.763769e+09,85088.00,85127.00,85028.00,85047.00,2.968572
7304794,1.763769e+09,85046.00,85046.00,84985.00,84989.00,0.400977
7304795,1.763769e+09,84989.00,85017.00,84983.00,85017.00,1.741910


## Обработка датасета

### Обработка временной метки

In [15]:
df['datetime'] = pd.to_datetime(df['Timestamp'], unit='s')
df.set_index('datetime', inplace=True)

In [16]:
df

,Timestamp,Open,High,Low,Close,Volume
datetime,,,,,,
2012-01-01 10:01:00,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
2012-01-01 10:02:00,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
2012-01-01 10:03:00,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
2012-01-01 10:04:00,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
2012-01-01 10:05:00,1.325412e+09,4.58,4.58,4.58,4.58,0.000000
...,...,...,...,...,...,...
2025-11-21 23:53:00,1.763769e+09,85065.00,85087.00,85031.00,85087.00,3.943960
2025-11-21 23:54:00,1.763769e+09,85088.00,85127.00,85028.00,85047.00,2.968572
2025-11-21 23:55:00,1.763769e+09,85046.00,85046.00,84985.00,84989.00,0.400977


### Пропуски

In [17]:
df.isnull().sum()

Timestamp    0
Open         0
High         0
Low          0
Close        0
Volume       0
dtype: int64

### Ресемплирование (по дням)

In [18]:
df_daily = df['Close'].resample('D').last().to_frame()

## Модель

### Prophet

In [19]:
prophet_df = df_daily[['Close']].reset_index()
prophet_df.columns = ['ds', 'y']
prophet_df = prophet_df.dropna()

In [22]:
prophet_model = Prophet()
prophet_model.fit(prophet_df)

20:40:57 - cmdstanpy - INFO - Chain [1] start processing
20:40:59 - cmdstanpy - INFO - Chain [1] done processing


Предсказание на месяц вперед

In [23]:
future_dates = prophet_model.make_future_dataframe(periods=30)
prophet_forecast = prophet_model.predict(future_dates)

In [24]:
prophet_predictions = prophet_forecast[['ds', 'yhat']].set_index('ds')

In [25]:
prophet_predictions

,yhat
ds,
2012-01-01,669.468536
2012-01-02,731.854343
2012-01-03,731.183918
2012-01-04,845.550864
2012-01-05,796.242054
...,...
2025-12-17,122487.588019
2025-12-18,122480.234435
2025-12-19,122530.523198


### LSTM (Long Short-Term Memory)

In [26]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_daily[['Close']].values)

In [28]:
def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        target = data[i + seq_length]
        sequences.append(seq)
        targets.append(target)
    return torch.FloatTensor(sequences), torch.FloatTensor(targets)

sequence_length = 30
X, y = create_sequences(np.array(scaled_data), sequence_length)

Определение модели

In [29]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=50, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq)
        predictions = self.linear(lstm_out[:, -1])
        return predictions

In [30]:
lstm_model = LSTMModel()

Обучение

In [31]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    lstm_model.train()
    optimizer.zero_grad()
    y_pred = lstm_model(X)
    loss = loss_function(y_pred, y)
    loss.backward()
    optimizer.step()

Прогноз

In [32]:
lstm_model.eval()
with torch.no_grad():
    lstm_scaled_predictions = lstm_model(X)
    lstm_predictions = scaler.inverse_transform(lstm_scaled_predictions.numpy())

lstm_dates = df_daily.index[sequence_length:]
lstm_results_df = pd.DataFrame(lstm_predictions, index=lstm_dates, columns=['LSTM_Pred'])

### Гибридный прогноз

Подготовка

In [33]:
# Выравниваем прогнозы Prophet
aligned_prophet_pred = prophet_predictions.loc[df_daily.index]['yhat']

residuals = df_daily['Close'] - aligned_prophet_pred
residuals = residuals.dropna()

Обучение LSTM на остатках

In [34]:
scaled_residuals = scaler.fit_transform(residuals.values.reshape(-1, 1))
X_res, y_res = create_sequences(scaled_residuals, sequence_length)

lstm_residuals_model = LSTMModel()

lstm_residuals_model.eval()
with torch.no_grad():
    predicted_residuals_scaled = lstm_residuals_model(X_res)
    predicted_residuals = scaler.inverse_transform(predicted_residuals_scaled.numpy())

Прогноз

In [35]:
prophet_for_hybrid = aligned_prophet_pred[sequence_length: sequence_length + len(predicted_residuals)]

hybrid_predictions = prophet_for_hybrid.values + predicted_residuals.flatten()

hybrid_dates = residuals.index[sequence_length: sequence_length + len(predicted_residuals)]
hybrid_results_df = pd.DataFrame(hybrid_predictions, index=hybrid_dates, columns=['Hybrid_Pred'])

## Результаты

In [37]:
results_df = df_daily[['Close']].copy()  # Исходные цены
results_df = results_df.join(prophet_predictions[['yhat']].rename(columns={'yhat': 'Prophet_Pred'}))
results_df = results_df.join(lstm_results_df)
results_df = results_df.join(hybrid_results_df)

results_df.tail()

,Close,Prophet_Pred,LSTM_Pred,Hybrid_Pred
datetime,,,,
2025-11-17,92102.0,117488.764540,100837.054688,82482.233290
2025-11-18,92911.0,117648.472324,99813.328125,82644.105137
2025-11-19,91474.0,117933.017175,98961.437500,82928.997644
2025-11-20,86527.0,118064.001156,98086.390625,83061.516781
2025-11-21,85091.0,118256.667153,96828.820312,83256.604653
